# Homework 3 -- Cloropleth map of grants in Swiss universities

In [27]:
# imports and all that
import pandas as pd

### Data wrangling

In [30]:
grants = pd.read_csv("data/P3_GrantExport.csv", sep=';', index_col=0)
grants.size
grants.shape

(48574, 8)

We'll try to reduce the dataset by removing the entries that are not in CH and irrelevant columns.
We also remove data where location cannot be inferred, i.e. which are missing both the institute and the university

These changes can be safely persisted in the file, as it will not influence the data we work with.

In [31]:
grants = grants[grants.University.notnull()]
grants = grants.drop(axis=1, errors='ignore', labels=['Project Title', 'Project Title English',
            'Responsible Applicant', 'Funding Instrument', 'Funding Instrument Hierarchy', 'Keywords'])

# drop rows where location cannot be inferred
filter_no_location = ~((grants.University == "Nicht zuteilbar - NA") & (grants.Institution.isnull()))
grants = grants[filter_no_location]
grants.dtypes
grants.shape

(48574, 8)

In [32]:
# Sum entries in the Approved Amount column are not numeric and such are being dropped
grants['Approved Amount'] = pd.to_numeric(grants['Approved Amount'], errors='coerce')
grants = grants[grants['Approved Amount'].isnull() == False]
grants.shape

(48464, 8)

In [33]:
# Run if you want to update the csv
grants.to_csv("data/P3_GrantExport.csv", sep=';') # keep the same sep

In [34]:
# Grouped by 'University' and the total sum each  
grants_grouped_by_uni = grants.groupby('University')
grants_grouped_by_uni = grants_grouped_by_uni.agg({'Approved Amount':'sum'})
grants_grouped_by_uni

,Approved Amount
University,
AO Research Institute - AORI,3.435621e+06
Allergie- und Asthmaforschung - SIAF,1.916996e+07
Berner Fachhochschule - BFH,3.102870e+07
Biotechnologie Institut Thurgau - BITG,2.492535e+06
Centre de rech. sur l'environnement alpin - CREALP,1.567678e+06
EPF Lausanne - EPFL,1.175316e+09
ETH Zürich - ETHZ,1.635597e+09
Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",4.836039e+07
